In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash
from dash import dcc, html, Input, Output, ctx
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from jupyter_dash import JupyterDash
import joblib
from dash import dcc, html, Input, Output, State, callback, ctx
import plotly.express as px
import pandas as pd
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Data Loading and Exploration

In [2]:
df = pd.read_csv("dataset.csv")
test_df = pd.read_csv("test.csv", on_bad_lines='skip')
val_df = pd.read_csv("val.csv", on_bad_lines='skip')

In [3]:
# viwing the traing data, test data and validation data
print("Training Data:")
display(df.head())
print("----------------------------------------------------------------------")
print("Testing Data:")
display(test_df.head())
print("----------------------------------------------------------------------")
print("Validation Data:")
display(val_df.head())

Training Data:


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,1.0,25.0,Female,Instagram,120.0,3.0,45.0,10.0,12.0,Happiness
1,2.0,30.0,Male,Twitter,90.0,5.0,20.0,25.0,30.0,Anger
2,3.0,22.0,Non-binary,Facebook,60.0,2.0,15.0,5.0,20.0,Neutral
3,4.0,28.0,Female,Instagram,200.0,8.0,100.0,30.0,50.0,Anxiety
4,5.0,33.0,Male,LinkedIn,45.0,1.0,5.0,2.0,10.0,Boredom


----------------------------------------------------------------------
Testing Data:


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,500.0,27.0,Female,Snapchat,120.0,4.0,40.0,18.0,22.0,Neutral
1,488.0,21.0,Non-binary,Snapchat,60.0,1.0,18.0,7.0,12.0,Neutral
2,776.0,28.0,Non-binary,Snapchat,115.0,3.0,38.0,18.0,27.0,Anxiety
3,869.0,27.0,Male,Telegram,105.0,3.0,48.0,20.0,28.0,Anxiety
4,573.0,21.0,Non-binary,Facebook,55.0,3.0,17.0,7.0,12.0,Neutral


----------------------------------------------------------------------
Validation Data:


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,10.0,31.0,Male,Instagram,170.0,5.0,80.0,20.0,35.0,Happiness
1,877.0,32.0,Female,Instagram,155.0,6.0,75.0,25.0,38.0,Happiness
2,230.0,26.0,Non-binary,Facebook,45.0,1.0,8.0,4.0,12.0,Sadness
3,876.0,28.0,Non-binary,Snapchat,115.0,3.0,38.0,18.0,27.0,Anxiety
4,376.0,28.0,Non-binary,Snapchat,115.0,3.0,38.0,18.0,27.0,Anxiety


In [4]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns in the training data.")
print(f"There are {test_df.shape[0]} rows and {test_df.shape[1]} columns in the testing data.")
print(f"There are {val_df.shape[0]} rows and {val_df.shape[1]} columns in the validation data.")

There are 1000 rows and 10 columns in the training data.
There are 103 rows and 10 columns in the testing data.
There are 148 rows and 10 columns in the validation data.


In [5]:
# training data info
print("Training Data Info:")
df.info()

Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     1000 non-null   float64
 1   Age                         1000 non-null   float64
 2   Gender                      1000 non-null   object 
 3   Platform                    1000 non-null   object 
 4   Daily_Usage_Time (minutes)  1000 non-null   float64
 5   Posts_Per_Day               1000 non-null   float64
 6   Likes_Received_Per_Day      1000 non-null   float64
 7   Comments_Received_Per_Day   1000 non-null   float64
 8   Messages_Sent_Per_Day       1000 non-null   float64
 9   Dominant_Emotion            1000 non-null   object 
dtypes: float64(7), object(3)
memory usage: 78.3+ KB


In [6]:
# testing data info
print("Testing Data Info:")
test_df.info()

Testing Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     103 non-null    float64
 1   Age                         103 non-null    float64
 2   Gender                      103 non-null    object 
 3   Platform                    103 non-null    object 
 4   Daily_Usage_Time (minutes)  103 non-null    float64
 5   Posts_Per_Day               103 non-null    float64
 6   Likes_Received_Per_Day      103 non-null    float64
 7   Comments_Received_Per_Day   103 non-null    float64
 8   Messages_Sent_Per_Day       103 non-null    float64
 9   Dominant_Emotion            103 non-null    object 
dtypes: float64(7), object(3)
memory usage: 8.2+ KB


In [7]:
# validation data info
print("Validation Data Info:")
val_df.info()

Validation Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     148 non-null    float64
 1   Age                         148 non-null    float64
 2   Gender                      148 non-null    object 
 3   Platform                    148 non-null    object 
 4   Daily_Usage_Time (minutes)  148 non-null    float64
 5   Posts_Per_Day               148 non-null    float64
 6   Likes_Received_Per_Day      148 non-null    float64
 7   Comments_Received_Per_Day   148 non-null    float64
 8   Messages_Sent_Per_Day       148 non-null    float64
 9   Dominant_Emotion            148 non-null    object 
dtypes: float64(7), object(3)
memory usage: 11.7+ KB


In [8]:
# checking for null values
print("Training Data:")
display(df.isnull().sum())
print("----------------------------------------------------------------------")
print("Testing Data:")
display(test_df.isnull().sum())
print("----------------------------------------------------------------------")
print("Validation Data:")
display(val_df.isnull().sum())

Training Data:


User_ID                       0
Age                           0
Gender                        0
Platform                      0
Daily_Usage_Time (minutes)    0
Posts_Per_Day                 0
Likes_Received_Per_Day        0
Comments_Received_Per_Day     0
Messages_Sent_Per_Day         0
Dominant_Emotion              0
dtype: int64

----------------------------------------------------------------------
Testing Data:


User_ID                       0
Age                           0
Gender                        0
Platform                      0
Daily_Usage_Time (minutes)    0
Posts_Per_Day                 0
Likes_Received_Per_Day        0
Comments_Received_Per_Day     0
Messages_Sent_Per_Day         0
Dominant_Emotion              0
dtype: int64

----------------------------------------------------------------------
Validation Data:


User_ID                       0
Age                           0
Gender                        0
Platform                      0
Daily_Usage_Time (minutes)    0
Posts_Per_Day                 0
Likes_Received_Per_Day        0
Comments_Received_Per_Day     0
Messages_Sent_Per_Day         0
Dominant_Emotion              0
dtype: int64

In [9]:
# list of coloums in the training data
df.columns 

Index(['User_ID', 'Age', 'Gender', 'Platform', 'Daily_Usage_Time (minutes)',
       'Posts_Per_Day', 'Likes_Received_Per_Day', 'Comments_Received_Per_Day',
       'Messages_Sent_Per_Day', 'Dominant_Emotion'],
      dtype='object')

In [10]:
#Describe the training data
df.describe()

,User_ID,Age,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,27.510000,95.950000,3.321000,39.898000,15.611000,22.560000
std,288.819436,3.930057,38.850442,1.914582,26.393867,8.819493,8.516274
min,1.000000,21.000000,40.000000,1.000000,5.000000,2.000000,8.000000
25%,250.750000,24.000000,65.000000,2.000000,20.000000,8.000000,17.750000
50%,500.500000,27.500000,85.000000,3.000000,33.000000,14.000000,22.000000
75%,750.250000,30.250000,120.000000,4.000000,55.000000,22.000000,28.000000
max,1000.000000,35.000000,200.000000,8.000000,110.000000,40.000000,50.000000


Age Distribution

In [11]:
df['Age'].isnull().sum()

0

In [12]:
df['Age'].unique()

array([25., 30., 22., 28., 33., 21., 27., 24., 29., 31., 23., 26., 34.,
       35., 32.])

In [13]:
plt = px.histogram(df, x='Age', title='Age Distribution')
plt.show()

Gender Distribution

In [14]:
df['Gender'].value_counts()

Gender
Female        370
Male          360
Non-binary    270
Name: count, dtype: int64

In [15]:
plt = px.histogram(df, x='Gender', title='Gender Distribution')
plt.show()

Platform Distribution

In [16]:
df['Platform'].value_counts()

Platform
Instagram    250
Twitter      200
Facebook     190
LinkedIn     120
Whatsapp      80
Telegram      80
Snapchat      80
Name: count, dtype: int64

In [17]:
plt = px.histogram(df, x='Platform', title='Platform Distribution')
plt.show()

Emotion Distribution

In [18]:
df['Dominant_Emotion'].value_counts()

Dominant_Emotion
Happiness    200
Neutral      200
Anxiety      170
Sadness      160
Boredom      140
Anger        130
Name: count, dtype: int64

In [19]:
plt = px.pie(df, names='Dominant_Emotion', title='Dominant Emotion Distribution')
# adding the values to the pie section
plt.update_traces(textposition='inside', textinfo='percent+label')
plt.show()

# Model

In [126]:
# Training Dataset
df.isnull().sum()

User_ID                       0
Age                           0
Gender                        0
Platform                      0
Daily_Usage_Time (minutes)    0
Posts_Per_Day                 0
Likes_Received_Per_Day        0
Comments_Received_Per_Day     0
Messages_Sent_Per_Day         0
Dominant_Emotion              0
dtype: int64

In [127]:
# Define features and target
X_train = df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_train = df['Dominant_Emotion'] # Target

In [128]:
X_train.columns


Index(['Age', 'Gender', 'Platform', 'Daily_Usage_Time (minutes)',
       'Posts_Per_Day', 'Likes_Received_Per_Day', 'Comments_Received_Per_Day',
       'Messages_Sent_Per_Day'],
      dtype='object')

In [129]:
# Define preprocessing steps. Seeperating the numerical and categorical features
numeric_features = ['Daily_Usage_Time (minutes)', 'Age' ,'Posts_Per_Day', 'Likes_Received_Per_Day', 'Comments_Received_Per_Day', 'Messages_Sent_Per_Day']
categorical_features = ['Gender', 'Platform']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

In [130]:
X_test = test_df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_test = test_df['Dominant_Emotion'] # Target

##### Random Forest

In [131]:
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(random_state=42))])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

print("Random Forest Classifier Report:")
print(classification_report(y_test, y_pred_rf))

Random Forest Classifier Report:
              precision    recall  f1-score   support

       Anger       0.90      1.00      0.95         9
     Anxiety       1.00      0.95      0.98        22
     Boredom       0.94      0.94      0.94        16
   Happiness       0.93      1.00      0.97        14
     Neutral       1.00      0.96      0.98        28
     Sadness       0.93      0.93      0.93        14

    accuracy                           0.96       103
   macro avg       0.95      0.96      0.96       103
weighted avg       0.96      0.96      0.96       103



In [132]:
import joblib

# Save the model
joblib.dump(rf_pipeline, 'rf_pipeline.pkl')


['rf_pipeline.pkl']

In [133]:
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred_rf
})

# Save to CSV
results_df.to_csv('rf_predictions.csv', index=False)


##### XGBoost

In [101]:
X_train = df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_train = df['Dominant_Emotion'] # Target
X_test = test_df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_test = test_df['Dominant_Emotion'] # Target

In [107]:
label_encoder = LabelEncoder()
y_encoded_train = label_encoder.fit_transform(y_train)
y_encoded_test = label_encoder.fit_transform(y_test)

In [108]:

# XGBoost Classifier
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', XGBClassifier(random_state=42))])

xgb_pipeline.fit(X_train, y_encoded_train)
y_pred_xgb = xgb_pipeline.predict(X_test)

In [109]:
# Decode the predicted labels back to original string labels for the report
y_test_decoded = label_encoder.inverse_transform(y_encoded_test)
y_pred_xgb_decoded = label_encoder.inverse_transform(y_pred_xgb)

print("XGBoost Classifier Report:")
print(classification_report(y_test_decoded, y_pred_xgb_decoded))

XGBoost Classifier Report:
              precision    recall  f1-score   support

       Anger       1.00      1.00      1.00         9
     Anxiety       1.00      0.95      0.98        22
     Boredom       0.94      0.94      0.94        16
   Happiness       0.88      1.00      0.93        14
     Neutral       1.00      0.96      0.98        28
     Sadness       0.93      0.93      0.93        14

    accuracy                           0.96       103
   macro avg       0.96      0.96      0.96       103
weighted avg       0.96      0.96      0.96       103



In [110]:
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred_xgb_decoded
})

# Save to CSV
results_df.to_csv('xgb_predictions.csv', index=False)


##### Neural Network

In [112]:
# Define features and target
X_train = df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_train = df['Dominant_Emotion'] # Target
X_test = test_df.drop(columns=['Dominant_Emotion', 'User_ID']) # Features
y_test = test_df['Dominant_Emotion'] # Target

In [113]:
label_encoder = LabelEncoder()

y_encoded_train = label_encoder.fit_transform(y_train)
y_onehot_train = to_categorical(y_encoded_train)

y_encoded_test = label_encoder.fit_transform(y_test)
y_onehot_test = to_categorical(y_encoded_test)

In [114]:
#  Define preprocessing steps for features
numeric_features = ['Daily_Usage_Time (minutes)', 'Posts_Per_Day', 'Likes_Received_Per_Day', 'Comments_Received_Per_Day', 'Messages_Sent_Per_Day']
categorical_features = ['Age', 'Gender', 'Platform']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Preprocess the features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [118]:
# Build the neural network
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_onehot_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_onehot_train, epochs=50, batch_size=32, validation_split=0.1)

c:\Users\Hend-PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.1872 - loss: 1.8303 - val_accuracy: 0.4200 - val_loss: 1.5211
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3694 - loss: 1.5458 - val_accuracy: 0.5000 - val_loss: 1.3894
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4054 - loss: 1.4721 - val_accuracy: 0.5400 - val_loss: 1.2942
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4493 - loss: 1.3708 - val_accuracy: 0.6100 - val_loss: 1.1972
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5116 - loss: 1.3095 - val_accuracy: 0.6600 - val_loss: 1.1108
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5488 - loss: 1.2226 - val_accuracy: 0.6700 - val_loss: 1.0273
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5761 - loss: 1.1600 - val_accuracy: 0.7400 - val_loss: 0.9461
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6121 - loss: 1.0523 - val_accuracy: 0.7500 - val_loss

In [119]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_onehot_test)
print(f'Accuracy: {accuracy:.2f}')

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_onehot_test, axis=1)

# Decode the predicted and true labels back to original string labels for the report
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_test_labels = label_encoder.inverse_transform(y_test_classes)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9790 - loss: 0.1498
Accuracy: 0.97
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [120]:
# Print classification report
print("Neural Network Classifier Report:")
print(classification_report(y_test_labels, y_pred_labels))

Neural Network Classifier Report:
              precision    recall  f1-score   support

       Anger       0.90      1.00      0.95         9
     Anxiety       1.00      0.95      0.98        22
     Boredom       0.94      0.94      0.94        16
   Happiness       1.00      1.00      1.00        14
     Neutral       1.00      0.96      0.98        28
     Sadness       0.93      1.00      0.97        14

    accuracy                           0.97       103
   macro avg       0.96      0.98      0.97       103
weighted avg       0.97      0.97      0.97       103



In [121]:
pred_df = pd.DataFrame({
    'Actual': y_test_labels,
    'Predicted': y_pred_labels
})

# Save to CSV
pred_df.to_csv('nn_predictions.csv', index=False)

#### Models Comparison

In [30]:
rf_df = pd.read_csv('rf_predictions.csv')
xgb_df = pd.read_csv('xgb_predictions.csv')
nn_df = pd.read_csv('nn_predictions.csv')

In [31]:
def evaluate_model(df, model_name):
    y_true = df['Actual']
    y_pred = df['Predicted']
    
    print(f"--- {model_name} ---")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("\n")

In [32]:
evaluate_model(rf_df, "Random Forest")
evaluate_model(xgb_df, "XGBoost")
evaluate_model(nn_df, "Neural Network")


--- Random Forest ---
Accuracy: 0.970873786407767
Classification Report:
               precision    recall  f1-score   support

       Anger       1.00      1.00      1.00         9
     Anxiety       1.00      0.95      0.98        22
     Boredom       0.94      0.94      0.94        16
   Happiness       0.93      1.00      0.97        14
     Neutral       1.00      0.96      0.98        28
     Sadness       0.93      1.00      0.97        14

    accuracy                           0.97       103
   macro avg       0.97      0.98      0.97       103
weighted avg       0.97      0.97      0.97       103



--- XGBoost ---
Accuracy: 0.9611650485436893
Classification Report:
               precision    recall  f1-score   support

       Anger       1.00      1.00      1.00         9
     Anxiety       1.00      0.95      0.98        22
     Boredom       0.94      0.94      0.94        16
   Happiness       0.88      1.00      0.93        14
     Neutral       1.00      0.96      0.

**Therefore Random Boost Performed Better Than Neural Network and XGBoost**

In [33]:
def plot_confusion_matrix(df, model_name):
    y_true = df['Actual']
    y_pred = df['Predicted']
    cm = confusion_matrix(y_true, y_pred)
    labels = sorted(df['Actual'].unique())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap="Blues")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

plot_confusion_matrix(rf_df, "Random Forest")
plot_confusion_matrix(xgb_df, "XGBoost")
plot_confusion_matrix(nn_df, "Neural Network")


AttributeError: 'Figure' object has no attribute 'figure'

# Data Visualization

### Before Model

**Plot 1**

In [34]:
# Group by Platform and Dominant_Emotion and count
counts = df.groupby(['Platform', 'Dominant_Emotion']).size().reset_index(name='Count')

# Get top 3 emotions per platform
top_emotions_per_platform = (
    counts.sort_values(['Platform', 'Count'], ascending=[True, False])
    .groupby('Platform')
    .head(3)
)

# Unique platforms
platforms = top_emotions_per_platform['Platform'].unique()

# Create subplots
fig = make_subplots(rows=1, cols=len(platforms), subplot_titles=platforms)

# Add bars for each platform
for i, platform in enumerate(platforms):
    platform_data = top_emotions_per_platform[top_emotions_per_platform['Platform'] == platform]
    fig.add_trace(
        go.Bar(
            x=platform_data['Dominant_Emotion'],
            y=platform_data['Count'],
            showlegend=False
        ),
        row=1, col=i+1
    )
    fig.update_xaxes(title_text="Emotion", tickvals=platform_data['Dominant_Emotion'], row=1, col=i+1)
    fig.update_yaxes(title_text="Number of Users", row=1, col=1)  # Only on first subplot

# Set layout
fig.update_layout(
    title_text="Top 3 Dominant Emotions by Social Media Platform",
    height=500,
    width=1600
)

fig.show()


**Plot 2**

In [35]:
# Step 1: Define emojis and colors for emotions
emotion_info = {
    'Happiness': {'emoji': '😊', 'color': '#FFD700'},  # Gold
    'Neutral': {'emoji': '😐', 'color': '#A9A9A9'},    # Gray
    'Anxiety': {'emoji': '😰', 'color': '#FF7F50'},    # Coral
    'Sadness': {'emoji': '😢', 'color': '#1E90FF'},    # DodgerBlue
    'Boredom': {'emoji': '🥱', 'color': '#C0C0C0'},    # Silver
    'Anger': {'emoji': '😠', 'color': '#DC143C'}       # Crimson
}

# Step 2: Create time bins
bins = [0, 30, 60, 90, 120, 150, 180, df['Daily_Usage_Time (minutes)'].max()]
labels = ['0-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180+']
df['Time_Range'] = pd.cut(df['Daily_Usage_Time (minutes)'], bins=bins, labels=labels, right=False)

# Step 3: Group and calculate percentages
grouped = df.groupby(['Time_Range', 'Dominant_Emotion']).size().reset_index(name='Count')
total_per_range = grouped.groupby('Time_Range')['Count'].transform('sum')
grouped['Percentage'] = (grouped['Count'] / total_per_range) * 100

# Step 4: Add emojis to emotion labels
grouped['Labeled_Emotion'] = grouped['Dominant_Emotion'].apply(lambda emo: f"{emotion_info[emo]['emoji']} {emo}")
grouped['Color'] = grouped['Dominant_Emotion'].apply(lambda emo: emotion_info[emo]['color'])

# Step 5: Build plot with dropdown
fig = go.Figure()
time_ranges = grouped['Time_Range'].unique().tolist()

for time_range in time_ranges:
    df_range = grouped[grouped['Time_Range'] == time_range]
    fig.add_trace(go.Bar(
        x=df_range['Labeled_Emotion'],
        y=df_range['Percentage'],
        name=str(time_range),
        visible=(time_range == time_ranges[0]),
        text=df_range['Percentage'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=df_range['Color']
    ))

# Step 6: Add dropdown menu
dropdown_buttons = [
    dict(label=str(r),
         method='update',
         args=[{'visible': [r == tr for tr in time_ranges]},
               {'title': f"Dominant Emotions for Time Spent: {r}"}])
    for r in time_ranges
]

fig.update_layout(
    title=f"Dominant Emotions for Time Spent: {time_ranges[0]}",
    xaxis_title="Dominant Emotion",
    yaxis_title="Percentage of Users",
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    plot_bgcolor='white'
)

fig.show()


C:\Users\habib\AppData\Local\Temp\ipykernel_12312\899240204.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\habib\AppData\Local\Temp\ipykernel_12312\899240204.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Plot 3**

In [36]:
avg_posts = df.groupby(['Platform', 'Gender'])['Posts_Per_Day'].mean().reset_index()

genders = avg_posts['Gender'].unique()
platforms = avg_posts['Platform'].unique()

gender_colors = {
    'Male': 'steelblue',
    'Female': 'mediumvioletred',
    'Other': 'seagreen' 
}

fig = go.Figure()

for gender in genders:
    gender_data = avg_posts[avg_posts['Gender'] == gender]
    fig.add_trace(
        go.Bar(
            x=gender_data['Platform'],
            y=gender_data['Posts_Per_Day'],
            name=gender,
            marker_color=gender_colors.get(gender, 'gray'),
            text=gender_data['Posts_Per_Day'].round(2), 
            textposition='outside', 
            textfont=dict(color='black', size=12, family='Arial Black'),
            visible=(gender == 'Male')
        )
    )

buttons = []
for i, gender in enumerate(genders):
    visibility = [False] * len(genders)
    visibility[i] = True
    buttons.append(
        dict(
            label=gender,
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Average Posts Per Day by Platform - {gender}"}
            ]
        )
    )

fig.update_layout(
    title="Average Posts Per Day by Platform - Male",
    xaxis=dict(
        title="Platform",
        tickfont=dict(family='Arial', size=14, color='black')  # Bold platform names
    ),
    yaxis=dict(
        title="Avg Posts Per Day",
        tickfont=dict(family='Arial', size=12, color='black')
    ),
    width=800,
    height=500,
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()

**Plot 4**

In [37]:
# Count number of users per platform
platform_counts = df['Platform'].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Number of Users']

# Create initial figure (vertical bar chart)
fig = go.Figure()

# Vertical bar (column) trace
fig.add_trace(
    go.Bar(
        x=platform_counts['Platform'],
        y=platform_counts['Number of Users'],
        name='Column Plot',
        visible=True
    )
)

# Horizontal bar trace
fig.add_trace(
    go.Bar(
        x=platform_counts['Number of Users'],
        y=platform_counts['Platform'],
        orientation='h',
        name='Bar Plot',
        visible=False
    )
)

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Column Plot",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "Most Popular Platforms (Column Chart)",
                            "xaxis": {"title": "Platform"},
                            "yaxis": {"title": "Number of Users"}}]),
                dict(label="Bar Plot",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Most Popular Platforms (Bar Chart)",
                            "xaxis": {"title": "Number of Users"},
                            "yaxis": {"title": "Platform"}}])
            ]),
            direction="down"
        )
    ],
    title="Most Popular Platforms (Column Chart)",
    xaxis_title="Platform",
    yaxis_title="Number of Users"
)

fig.show()


**Plot 5**

In [38]:
# df = df.dropna(subset=['Dominant_Emotion', 'Likes_Received_Per_Day', 'Comments_Received_Per_Day'])

# Use MINTY Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

# Add custom CSS to hide the debug, error, and version bar at the bottom
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'  # Bootstrap styles
})

app.layout = dbc.Container([
    html.H3("Engagement vs Dominant Emotion", className="text-center my-4"),
    # Engagement (Likes & Comments) vs Dominant Emotion
    dcc.Dropdown(
        id='metric-select',
        options=[
            {'label': 'Likes Received Per Day', 'value': 'Likes'},
            {'label': 'Comments Received Per Day', 'value': 'Comments'},
            {'label': 'Both (Stacked)', 'value': 'Both'}
        ],
        value='Likes',  # Default value
        clearable=False,
        className="mb-4",
        style={'width': '50%'}
    ),
    
    dcc.Graph(id='emotion-engagement-plot')
], fluid=True)


# Callback to update graph
@app.callback(
    Output('emotion-engagement-plot', 'figure'),
    Input('metric-select', 'value')
)
def update_graph(selected_metric):
    if selected_metric == 'Likes':
        grouped = df.groupby('Dominant_Emotion')['Likes_Received_Per_Day'].mean().reset_index()
        fig = px.bar(
            grouped,
            x='Dominant_Emotion',
            y='Likes_Received_Per_Day',
            title="Average Likes Received per Emotion",
            labels={'Likes_Received_Per_Day': 'Average Likes'},
            color='Dominant_Emotion',
            color_discrete_sequence=px.colors.sequential.Mint
        )
    
    elif selected_metric == 'Comments':
        grouped = df.groupby('Dominant_Emotion')['Comments_Received_Per_Day'].mean().reset_index()
        fig = px.bar(
            grouped,
            x='Dominant_Emotion',
            y='Comments_Received_Per_Day',
            title="Average Comments Received per Emotion",
            labels={'Comments_Received_Per_Day': 'Average Comments'},
            color='Dominant_Emotion',
            color_discrete_sequence=px.colors.sequential.Oranges
        )
    
    else:  # Both (stacked)
        likes = df.groupby('Dominant_Emotion')['Likes_Received_Per_Day'].mean()
        comments = df.groupby('Dominant_Emotion')['Comments_Received_Per_Day'].mean()
        emotions = likes.index

        fig = go.Figure(data=[
            go.Bar(name='Likes', x=emotions, y=likes, marker_color='#66c2a5'),    # Minty-like color
            go.Bar(name='Comments', x=emotions, y=comments, marker_color='#fc8d62') # Complementary
        ])
        fig.update_layout(
            barmode='stack',
            title='Average Engagement per Emotion (Stacked)',
            xaxis_title='Dominant Emotion',
            yaxis_title='Average Engagement',
            template='plotly_white'
        )

    return fig


# Run the app without showing errors and server bar
if __name__ == '__main__':
    app.run(debug=False, suppress_callback_exceptions=True)


c:\Users\habib\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\resources.py:96: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



**Plot 6**

In [39]:
## Time spent (daily) over age groups (Multitasking habits: using social media while studying or working)

# Create a new column for Age Groups
age_bins = [21, 24, 29, 35]  # The bin edges for age groups
age_labels = ['21-24', '25-29', '30-35']  # Labels for the age groups
df['Age_Group'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=True)

# Group by Age_Group and calculate average daily usage time
age_grouped = df.groupby('Age_Group')['Daily_Usage_Time (minutes)'].mean().reset_index()

# Create bar plot
fig = px.bar(
    age_grouped,
    x='Age_Group',
    y='Daily_Usage_Time (minutes)',
    title="Average Daily Time Spent on Social Media by Age Group",
    labels={'Daily_Usage_Time (minutes)': 'Average Daily Usage Time (minutes)', 'Age_Group': 'Age Group'},
    color='Age_Group',  # Color by age group for better visual distinction
    color_discrete_sequence=px.colors.sequential.Mint  # Optional: change color scheme
)



fig.show()


C:\Users\habib\AppData\Local\Temp\ipykernel_12312\1797914549.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### After Model

In [40]:
df_test = pd.read_csv('test.csv')
rf_preds = pd.read_csv('rf_predictions.csv')
df_test['Predicted_Emotion'] = rf_preds['Predicted']

**Plot 1**

In [41]:
# Count unique predicted emotions per platform
emotion_counts = df_test.groupby('Platform')['Predicted_Emotion'].nunique()
print(emotion_counts)


Platform
Facebook     4
Instagram    4
LinkedIn     3
Snapchat     4
Telegram     4
Twitter      6
Whatsapp     2
Name: Predicted_Emotion, dtype: int64


In [42]:
# Group by Platform and Predicted_Emotion
pred_counts = (
    df_test.groupby(['Platform', 'Predicted_Emotion'])
    .size()
    .reset_index(name='Count')
    .rename(columns={'Predicted_Emotion': 'Emotion'})
)

# Get top 2 predicted emotions per platform
top_preds_per_platform = (
    pred_counts.sort_values(['Platform', 'Count'], ascending=[True, False])
    .groupby('Platform')
    .head(2)
)

# Get unique platforms
platforms = top_preds_per_platform['Platform'].unique()

# Create subplots
fig = make_subplots(rows=1, cols=len(platforms), subplot_titles=platforms)

# Add bars for each platform
for i, platform in enumerate(platforms):
    platform_data = top_preds_per_platform[top_preds_per_platform['Platform'] == platform]
    fig.add_trace(
        go.Bar(
            x=platform_data['Emotion'],
            y=platform_data['Count'],
            marker_color='orange',
            showlegend=False
        ),
        row=1, col=i+1
    )
    fig.update_xaxes(title_text="Emotion", tickvals=platform_data['Emotion'], row=1, col=i+1)
    fig.update_yaxes(title_text="Predicted Count" if i == 0 else "", row=1, col=i+1)

# Set layout
fig.update_layout(
    title_text="Top 3 Predicted Emotions by Social Media Platform (Random Forest)",
    height=500,
    width=1400,
    margin=dict(t=50)
)

fig.show()


**Plot 2**

In [43]:
# Define emojis and colors for emotions
emotion_info = {
    'Happiness': {'emoji': '😊', 'color': '#FFD700'},
    'Neutral': {'emoji': '😐', 'color': '#A9A9A9'},
    'Anxiety': {'emoji': '😰', 'color': '#FF7F50'},
    'Sadness': {'emoji': '😢', 'color': '#1E90FF'},
    'Boredom': {'emoji': '🥱', 'color': '#C0C0C0'},
    'Anger': {'emoji': '😠', 'color': '#DC143C'}
}

# Create time bins
bins = [0, 30, 60, 90, 120, 150, 180, df_test['Daily_Usage_Time (minutes)'].max()]
labels = ['0-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180+']
df_test['Time_Range'] = pd.cut(df_test['Daily_Usage_Time (minutes)'], bins=bins, labels=labels, right=False)

# Group by Time_Range and Predicted_Emotion
grouped = df_test.groupby(['Time_Range', 'Predicted_Emotion']).size().reset_index(name='Count')
total_per_range = grouped.groupby('Time_Range')['Count'].transform('sum')
grouped['Percentage'] = (grouped['Count'] / total_per_range) * 100

# Add emoji and color
grouped['Labeled_Emotion'] = grouped['Predicted_Emotion'].apply(lambda emo: f"{emotion_info[emo]['emoji']} {emo}")
grouped['Color'] = grouped['Predicted_Emotion'].apply(lambda emo: emotion_info[emo]['color'])

# Create the plot
fig = go.Figure()
time_ranges = grouped['Time_Range'].unique().tolist()

for time_range in time_ranges:
    df_range = grouped[grouped['Time_Range'] == time_range]
    fig.add_trace(go.Bar(
        x=df_range['Labeled_Emotion'],
        y=df_range['Percentage'],
        name=str(time_range),
        visible=(time_range == time_ranges[0]),
        text=df_range['Percentage'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=df_range['Color']
    ))

# Dropdown menu
dropdown_buttons = [
    dict(label=str(r),
         method='update',
         args=[{'visible': [r == tr for tr in time_ranges]},
               {'title': f"Predicted Emotions for Time Spent: {r}"}])
    for r in time_ranges
]

fig.update_layout(
    title=f"Predicted Emotions for Time Spent: {time_ranges[0]}",
    xaxis_title="Emotion",
    yaxis_title="Percentage of Users",
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    plot_bgcolor='white'
)

fig.show()


C:\Users\habib\AppData\Local\Temp\ipykernel_12312\129241045.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\habib\AppData\Local\Temp\ipykernel_12312\129241045.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Plot 3**

In [44]:
# Now use df_test (filtered test data with predicted labels) for plotting
avg_posts = df_test.groupby(['Platform', 'Gender'])['Posts_Per_Day'].mean().reset_index()

platforms = avg_posts['Platform'].unique()

ordered_genders = ['Male', 'Female', 'Non-binary']

gender_colors = {
    'Male': 'steelblue',
    'Female': 'mediumvioletred',
    'Other': 'seagreen'
}

fig = go.Figure()

for gender in ordered_genders:
    gender_data = avg_posts[avg_posts['Gender'] == gender]
    fig.add_trace(
        go.Bar(
            x=gender_data['Platform'],
            y=gender_data['Posts_Per_Day'],
            name=gender,
            marker_color=gender_colors.get(gender, 'gray'),
            text=gender_data['Posts_Per_Day'].round(2),
            textposition='outside',
            textfont=dict(color='black', size=12, family='Arial Black'),
            visible=(gender == 'Male')  # Default visibility
        )
    )

# Create buttons for dropdown
buttons = []
for i, gender in enumerate(ordered_genders):
    visibility = [False] * len(genders)
    visibility[i] = True
    buttons.append(
        dict(
            label=gender,
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Avg Posts Per Day by Platform (Predicted) - {gender}"}
            ]
        )
    )

# Layout
fig.update_layout(
    title="Avg Posts Per Day by Platform (Predicted) - Male",
    xaxis=dict(
        title="Platform",
        tickfont=dict(family='Arial', size=14, color='black')
    ),
    yaxis=dict(
        title="Avg Posts Per Day",
        tickfont=dict(family='Arial', size=12, color='black')
    ),
    width=800,
    height=500,
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()


**Plot 4**

In [45]:
# Count number of users per platform (based on test set)
platform_counts = df_test['Platform'].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Number of Users']

# Create figure with both vertical and horizontal bar chart options
fig = go.Figure()

# Vertical bar chart
fig.add_trace(
    go.Bar(
        x=platform_counts['Platform'],
        y=platform_counts['Number of Users'],
        name='Column Plot',
        visible=True
    )
)

# Horizontal bar chart
fig.add_trace(
    go.Bar(
        x=platform_counts['Number of Users'],
        y=platform_counts['Platform'],
        orientation='h',
        name='Bar Plot',
        visible=False
    )
)

# Add dropdown menu for toggling
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=[
                dict(label="Column Plot",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "Most Popular Platforms (Predicted) - Column Chart",
                            "xaxis": {"title": "Platform"},
                            "yaxis": {"title": "Number of Users"}}]),
                dict(label="Bar Plot",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Most Popular Platforms (Predicted) - Bar Chart",
                            "xaxis": {"title": "Number of Users"},
                            "yaxis": {"title": "Platform"}}])
            ],
            direction="down",
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top"
        )
    ],
    title="Most Popular Platforms (Predicted) - Column Chart",
    xaxis_title="Platform",
    yaxis_title="Number of Users"
)

fig.show()


**Plot 5**

In [46]:
likes_grouped = df_test.groupby('Predicted_Emotion')['Likes_Received_Per_Day'].mean()
comments_grouped = df_test.groupby('Predicted_Emotion')['Comments_Received_Per_Day'].mean()
emotions = likes_grouped.index

# Create figure and add all traces (some hidden by default)
fig = go.Figure()

# Trace 0: Likes only
fig.add_trace(
    go.Bar(
        x=emotions,
        y=likes_grouped,
        name='Likes',
        marker_color='#66c2a5',
        visible=True
    )
)

# Trace 1: Comments only
fig.add_trace(
    go.Bar(
        x=emotions,
        y=comments_grouped,
        name='Comments',
        marker_color='#fc8d62',
        visible=False
    )
)

# Traces 2 and 3: Both (stacked)
fig.add_trace(
    go.Bar(
        x=emotions,
        y=likes_grouped,
        name='Likes (Stacked)',
        marker_color='#66c2a5',
        visible=False
    )
)

fig.add_trace(
    go.Bar(
        x=emotions,
        y=comments_grouped,
        name='Comments (Stacked)',
        marker_color='#fc8d62',
        visible=False
    )
)

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=[
                dict(
                    label="Likes",
                    method="update",
                    args=[{"visible": [True, False, False, False]},
                          {"title": "Average Likes Received per Predicted Emotion",
                           "barmode": "group",
                           "xaxis": {"title": "Predicted Emotion"},
                           "yaxis": {"title": "Average Likes"}}]
                ),
                dict(
                    label="Comments",
                    method="update",
                    args=[{"visible": [False, True, False, False]},
                          {"title": "Average Comments Received per Predicted Emotion",
                           "barmode": "group",
                           "xaxis": {"title": "Predicted Emotion"},
                           "yaxis": {"title": "Average Comments"}}]
                ),
                dict(
                    label="Both (Stacked)",
                    method="update",
                    args=[{"visible": [False, False, True, True]},
                          {"title": "Average Engagement per Predicted Emotion (Stacked)",
                           "barmode": "stack",
                           "xaxis": {"title": "Predicted Emotion"},
                           "yaxis": {"title": "Average Engagement"}}]
                ),
            ],
            direction="down",
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top"
        )
    ],
    title="Average Likes Received per Predicted Emotion",
    xaxis_title="Predicted Emotion",
    yaxis_title="Average Likes"
)

fig.show()


**Plot 6**

In [47]:
# Create a new column for Age Groups
age_bins = [21, 24, 29, 35]  # The bin edges for age groups
age_labels = ['21-24', '25-29', '30-35']  # Labels for the age groups
df_test['Age_Group'] = pd.cut(df_test['Age'], bins=age_bins, labels=age_labels, right=True)

# Group by Age_Group and Predicted_Emotion and calculate average daily usage time
age_grouped = df_test.groupby(['Age_Group', 'Predicted_Emotion'])['Daily_Usage_Time (minutes)'].mean().reset_index()

# Create bar plot
fig = px.bar(
    age_grouped,
    x='Age_Group',
    y='Daily_Usage_Time (minutes)',
    color='Predicted_Emotion',  # Color by predicted emotion for better visual distinction
    title="Average Daily Time Spent on Social Media by Age Group and Predicted Emotion",
    labels={'Daily_Usage_Time (minutes)': 'Average Daily Usage Time (minutes)', 'Age_Group': 'Age Group'},
    color_discrete_sequence=px.colors.sequential.Mint  # Optional: change color scheme
)

fig.show()


C:\Users\habib\AppData\Local\Temp\ipykernel_12312\596776577.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# Dashboard

**Plot 1**

In [51]:
counts = df.groupby(['Platform', 'Dominant_Emotion']).size().reset_index(name='Count')

# Get top 3 emotions per platform
top_emotions_per_platform = (
    counts.sort_values(['Platform', 'Count'], ascending=[True, False])
    .groupby('Platform')
    .head(3)
)

# Unique platforms
platforms = top_emotions_per_platform['Platform'].unique()

# Create subplots
fig1 = make_subplots(rows=1, cols=len(platforms), subplot_titles=platforms)

# Add bars for each platform
for i, platform in enumerate(platforms):
    platform_data = top_emotions_per_platform[top_emotions_per_platform['Platform'] == platform]
    fig1.add_trace(
        go.Bar(
            x=platform_data['Dominant_Emotion'],
            y=platform_data['Count'],
            showlegend=False
        ),
        row=1, col=i+1
    )
    fig1.update_xaxes(title_text="Emotion", tickvals=platform_data['Dominant_Emotion'], row=1, col=i+1)
    fig1.update_yaxes(title_text="Number of Users", row=1, col=1)  # Only on first subplot

# Set layout
fig1.update_layout(
    title={
        'text': "<b>Top 3 Dominant Emotions by Social Media Platform</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    },
    height=400,
    width=1200,
    margin=dict(t=80)
)

**Plot 2**

In [52]:
# Step 1: Define emojis and colors for emotions
emotion_info = {
    'Happiness': {'emoji': '😊', 'color': '#FFD700'},
    'Neutral': {'emoji': '😐', 'color': '#A9A9A9'},
    'Anxiety': {'emoji': '😰', 'color': '#FF7F50'},
    'Sadness': {'emoji': '😢', 'color': '#1E90FF'},
    'Boredom': {'emoji': '🥱', 'color': '#C0C0C0'},
    'Anger': {'emoji': '😠', 'color': '#DC143C'}
}

# Step 2: Create time bins
bins = [0, 30, 60, 90, 120, 150, 180, df['Daily_Usage_Time (minutes)'].max()]
labels = ['0-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180+']
df['Time_Range'] = pd.cut(df['Daily_Usage_Time (minutes)'], bins=bins, labels=labels, right=False)

# Step 3: Group and calculate percentages
grouped = df.groupby(['Time_Range', 'Dominant_Emotion']).size().reset_index(name='Count')
total_per_range = grouped.groupby('Time_Range')['Count'].transform('sum')
grouped['Percentage'] = (grouped['Count'] / total_per_range) * 100

# Step 4: Add emojis and colors
grouped['Labeled_Emotion'] = grouped['Dominant_Emotion'].apply(lambda emo: f"{emotion_info[emo]['emoji']} {emo}")
grouped['Color'] = grouped['Dominant_Emotion'].apply(lambda emo: emotion_info[emo]['color'])

# Step 5: Build plot with dropdown
fig2 = go.Figure()
time_ranges = grouped['Time_Range'].unique().tolist()

for time_range in time_ranges:
    df_range = grouped[grouped['Time_Range'] == time_range]
    fig2.add_trace(go.Bar(
        x=df_range['Labeled_Emotion'],
        y=df_range['Percentage'],
        name=str(time_range),
        visible=(time_range == '30-60'),
        text=df_range['Percentage'].round(1).astype(str) + '%',
        textposition='auto',
        marker_color=df_range['Color']
    ))

# Step 6: Dropdown menu
dropdown_buttons = [
    dict(label=str(r),
         method='update',
         args=[{'visible': [r == tr for tr in time_ranges]},
              {'title': {
                'text': f"<b>Dominant Emotions ({r})</b>",
                'font': {'size': 24},  
                'x': 0.5,
                'xanchor': 'center'
            }}])
    for r in time_ranges
]

fig2.update_layout(
    title={
        'text': "<b>Dominant Emotions (30–60 min)</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Dominant Emotion",
    yaxis_title="Percentage of Users",
    updatemenus=[dict(
        active=time_ranges.index('30-60'),
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    plot_bgcolor='white',
    height=500,
    width=800
)


C:\Users\Hend-PC\AppData\Local\Temp\ipykernel_6824\2091750928.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Hend-PC\AppData\Local\Temp\ipykernel_6824\2091750928.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Plot 3**

In [53]:
avg_posts = df.groupby(['Platform', 'Gender'])['Posts_Per_Day'].mean().reset_index()

genders = ['Male'] + [g for g in avg_posts['Gender'].unique() if g != 'Male']
platforms = avg_posts['Platform'].unique()

gender_colors = {
    'Male': 'steelblue',
    'Female': 'mediumvioletred',
    'Other': 'seagreen'
}

fig3 = go.Figure()

# Add traces for each gender
for gender in genders:
    gender_data = avg_posts[avg_posts['Gender'] == gender]
    fig3.add_trace(
        go.Bar(
            x=gender_data['Platform'],
            y=gender_data['Posts_Per_Day'],
            name=gender,
            marker_color=gender_colors.get(gender, 'gray'),
            text=gender_data['Posts_Per_Day'].round(2),
            textposition='outside',
            textfont=dict(color='black', size=12, family='Arial Black'),
            visible=(gender == 'Male')  # Only 'Male' visible by default
        )
    )

# Dropdown menu
buttons = []
for i, gender in enumerate(genders):
    visibility = [False] * len(genders)
    visibility[i] = True
    buttons.append(
        dict(
            label=gender,
            method="update",
            args=[
                {"visible": visibility}
            ]
        )
    )

fig3.update_layout(
    title={
        'text': "<b>Average Daily Posts by Platform</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title="Platform",
        tickfont=dict(family='Arial', size=14, color='black')
    ),
    yaxis=dict(
        title="Avg Posts Per Day",
        tickfont=dict(family='Arial', size=12, color='black')
    ),
    width=500,
    height=500,
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor='white'
)


**Plot 4**

In [54]:
# === Plot 4 Code ===

# Count users per platform
platform_counts = df['Platform'].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Number of Users']

# Create figure with two bar orientations
fig4 = go.Figure()

# Column chart (vertical bars)
fig4.add_trace(
    go.Bar(
        x=platform_counts['Platform'],
        y=platform_counts['Number of Users'],
        name='Column Plot',
        visible=True
    )
)

# Horizontal bar chart
fig4.add_trace(
    go.Bar(
        x=platform_counts['Number of Users'],
        y=platform_counts['Platform'],
        orientation='h',
        name='Bar Plot',
        visible=False
    )
)

# Dropdown toggle
fig4.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Column Plot",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"xaxis": {"title": "Platform"},
                            "yaxis": {"title": "Number of Users"}}]),
                dict(label="Bar Plot",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"xaxis": {"title": "Number of Users"},
                            "yaxis": {"title": "Platform"}}])
            ]),
            direction="down",
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top"
        )
    ],
    title={
        'text': "<b>Most Popular Platforms</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Platform",
    yaxis_title="Number of Users",
    plot_bgcolor='white',
    height=400,
    width=800
)

#


**Plot 5**

In [55]:
import plotly.graph_objects as go

# Prepare data for Plot 5 dropdown figure

likes = df.groupby('Dominant_Emotion')['Likes_Received_Per_Day'].mean()
comments = df.groupby('Dominant_Emotion')['Comments_Received_Per_Day'].mean()
emotions = likes.index.tolist()

# Create traces for Likes, Comments, and Both stacked (both bars together)

trace_likes = go.Bar(
    x=emotions,
    y=likes,
    name='Likes',
    marker_color='#66c2a5',
    visible=True
)

trace_comments = go.Bar(
    x=emotions,
    y=comments,
    name='Comments',
    marker_color='#fc8d62',
    visible=False
)

# For "Both" stacked, we will add both Likes + Comments traces visible
trace_both_likes = go.Bar(
    x=emotions,
    y=likes,
    name='Likes',
    marker_color='#66c2a5',
    visible=False
)

trace_both_comments = go.Bar(
    x=emotions,
    y=comments,
    name='Comments',
    marker_color='#fc8d62',
    visible=False
)

fig5 = go.Figure(data=[trace_likes, trace_comments, trace_both_likes, trace_both_comments])

# Define buttons for dropdown
buttons = [
    dict(
        label='Likes',
        method='update',
        args=[{'visible': [True, False, False, False]},
              {'title': 'Average Likes Received per Emotion',
               'barmode': 'group',
               'yaxis': {'title': 'Average Likes'},
               'xaxis': {'title': 'Dominant Emotion'}}]
    ),
    dict(
        label='Comments',
        method='update',
        args=[{'visible': [False, True, False, False]},
              {'title': 'Average Comments Received per Emotion',
               'barmode': 'group',
               'yaxis': {'title': 'Average Comments'},
               'xaxis': {'title': 'Dominant Emotion'}}]
    ),
    dict(
        label='Both (Stacked)',
        method='update',
        args=[{'visible': [False, False, True, True]},
              {'title': 'Average Engagement per Emotion (Stacked)',
               'barmode': 'stack',
               'yaxis': {'title': 'Average Engagement'},
               'xaxis': {'title': 'Dominant Emotion'}}]
    ),
]

fig5.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=0.5,
            xanchor='center',
            y=1.15,
            yanchor='top'
        )
    ],
    template='plotly_white',
    height=400,
    width=1200,
    title={
        'text': "<b>Average Likes Received per Emotion</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'}
)


**Plot 6**

In [56]:
# Create Age_Group column with bins and labels
age_bins = [21, 24, 29, 35]
age_labels = ['21-24', '25-29', '30-35']
df['Age_Group'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=True)

# Group by Age_Group and calculate average daily usage time
age_grouped = df.groupby('Age_Group')['Daily_Usage_Time (minutes)'].mean().reset_index()

# Create bar plot figure
fig6 = px.bar(
    age_grouped,
    x='Age_Group',
    y='Daily_Usage_Time (minutes)',
    title="Average Daily Time Spent on Social Media by Age Group",
    labels={
        'Daily_Usage_Time (minutes)': 'Average Daily Usage Time (minutes)',
        'Age_Group': 'Age Group'
    },
    color='Age_Group',
    color_discrete_sequence=px.colors.sequential.Mint
)

fig6.update_layout(
    height=400,
    width=600,
    plot_bgcolor='white',
    title={
        'text': "<b>Average Social Media Use by Age</b>",
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    }
)

# To show the figure standalone (outside Dash)
# fig6.show()


C:\Users\Hend-PC\AppData\Local\Temp\ipykernel_6824\867796317.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [57]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

age_group_items = [
    dbc.DropdownMenuItem("All Ages", id="age-all", n_clicks=1),
    dbc.DropdownMenuItem("From 21 to 24", id="age-21-24", n_clicks=0),
    dbc.DropdownMenuItem("From 25 to 29", id="age-25-29", n_clicks=0),
    dbc.DropdownMenuItem("From 30 to 35", id="age-30-35", n_clicks=0),
]

c:\Users\Hend-PC\anaconda3\Lib\site-packages\dash\dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [58]:
navbar = dbc.Navbar(
    html.Div(
        [
            html.Div(style={"flex": "1"}),  
            dbc.NavbarBrand(
                html.Span(
                    "Emotion Analysis Dashboard",
                    style={'fontWeight': 'bold', 'fontSize': '50px', 'color': 'white'}
                ),
                href="#",
                style={"margin": "0"}
            ),
            html.Div(
                dbc.DropdownMenu(
                    children=age_group_items,
                    nav=True,
                    in_navbar=True,
                    label="Age Group",
                    id="age-dropdown",
                    style={
                        "color": "white",
                        "fontWeight": "bold", 
                        "fontSize": "24px"
                    }
                ),
                style={"flex": "1", "display": "flex", "justifyContent": "flex-end"}
            ),  
        ],
        style={
            'width': '100%',
            'display': 'flex',
            'justifyContent': 'space-between',
            'alignItems': 'center',
            'paddingLeft': '20px',
            'paddingRight': '20px',
            'margin': '0 auto'
        }
    ),
    color="primary",
    dark=True,
    sticky="top",
    style={
        "width": "100%", 
        "maxWidth": "2680px",  
        "margin": "0 auto",  
        "padding": "0"
    }
)

In [59]:
buttons = html.Div([
    dbc.Button(
        "Home Page",
        id="home-page-btn",
        color="primary",
        size="lg",
        style={
            "marginRight": "20px",
            "fontWeight": "bold",
            "fontSize": "22px",
            "padding": "12px 30px"
        }
    ),
    dbc.Button(
        "Test Your Emotion",
        id="test-emotion-btn",
        color="secondary",
        size="lg",
        style={
            "fontWeight": "bold",
            "fontSize": "22px",
            "padding": "12px 30px"
        }
    )
], style={
    'display': 'flex',
    'justifyContent': 'center',
    'alignItems': 'center',
    'marginTop': '30px',
    'marginBottom': '30px'
})

In [60]:
def get_home_content(fig1, fig2, fig3, fig4, fig5, fig6):
    return html.Div([
        html.Div([
            html.Div(dcc.Graph(id='emotions-by-usage-time', figure=fig2), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
            html.Div(dcc.Graph(id='top-emotions-platform', figure=fig1), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
            html.Div(dcc.Graph(id='avg-posts-platform-gender', figure=fig3), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
            html.Div(dcc.Graph(id='platform-popularity-toggle', figure=fig4), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
            html.Div(dcc.Graph(id='emotion-engagement-plot', figure=fig5), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
            html.Div(dcc.Graph(id='daily-usage-by-age', figure=fig6), 
                    style={'padding': '5px', 'border': '2px solid #ccc', 'borderRadius': '2px','boxShadow': '0 4px 8px rgba(0, 0, 0, 0.15)'}),
        ], style={
            'display': 'grid',
            'gridTemplateColumns': 'repeat(3, 1fr)',
            'gridGap': '10px',
            'padding': '0 20px',
            'justifyContent': 'center'
        })
    ])

In [61]:
rf_pipeline = joblib.load("rf_pipeline.pkl")

c:\Users\Hend-PC\anaconda3\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning:

Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

c:\Users\Hend-PC\anaconda3\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning:

Trying to unpickle estimator OneHotEncoder from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

c:\Users\Hend-PC\anaconda3\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning:

Trying to unpickle estimator ColumnTransformer from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid resu

In [62]:
# Test emotion page content
def get_test_emotion_content():
    return html.Div([
        html.H2(
            "🎯 Predict Your Dominant Emotion Based on Social Media Usage",
            style={
                'textAlign': 'center',
                'color': '#3AAFA9',
                'marginBottom': '40px',
                'fontWeight': 'bold'
            }
        ),

        html.Div([
            # Left Column
            html.Div([
                html.Label("Age", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='age', type='number', min=10, max=100,
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

                html.Label("Gender", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Dropdown(id='gender', options=[
                    {'label': 'Male', 'value': 'Male'},
                    {'label': 'Female', 'value': 'Female'}
                ], style={'borderRadius': '5px', 'marginBottom': '15px'}),

                html.Label("Mostly Used Platform", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Dropdown(id='platform', options=[
                    {'label': 'Instagram', 'value': 'Instagram'},
                    {'label': 'Facebook', 'value': 'Facebook'},
                    {'label': 'Twitter', 'value': 'Twitter'},
                    {'label': 'TikTok', 'value': 'TikTok'},
                    {'label': 'YouTube', 'value': 'YouTube'},
                    {'label': 'LinkedIn', 'value': 'LinkedIn'},
                    {'label': 'Snapchat', 'value': 'Snapchat'}
                ], style={'borderRadius': '5px', 'marginBottom': '15px'}),

                html.Label("Daily Usage Time (minutes)", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='daily', type='number',
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),
            ], style={
                'width': '48%',
                'display': 'inline-block',
                'verticalAlign': 'top',
                'paddingRight': '20px',
                'boxSizing': 'border-box'
            }),

            # Right Column
            html.Div([
                html.Label("Posts Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='posts', type='number',
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

                html.Label("Likes Received Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='likes', type='number',
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

                html.Label("Comments Received Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='comments', type='number',
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

                html.Label("Messages Sent Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
                dcc.Input(id='messages', type='number',
                          style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                                 'border': '1px solid #95E1D3', 'marginBottom': '15px'}),
            ], style={
                'width': '48%',
                'display': 'inline-block',
                'verticalAlign': 'top'
            }),
        ], style={
            'maxWidth': '1100px',
            'margin': 'auto',
            'backgroundColor': '#E6F7F6',
            'padding': '30px',
            'borderRadius': '12px',
            'boxShadow': '0 4px 12px rgba(0,0,0,0.1)',
            'marginBottom': '40px'
        }),

        html.Div([
            html.Button("Predict Emotion", id='predict-button', n_clicks=0,
                        style={
                            'backgroundColor': '#3AAFA9',
                            'color': 'white',
                            'border': 'none',
                            'padding': '14px 34px',
                            'borderRadius': '25px',
                            'cursor': 'pointer',
                            'fontSize': '16px',
                            'fontWeight': '600',
                            'display': 'block',
                            'margin': '0 auto'
                        }),
        ], style={'marginBottom': '40px'}),

        html.Div(id='prediction-output',
                 style={'fontSize': 24, 'textAlign': 'center', 'color': '#17252A', 'fontWeight': 'bold', 'minHeight': '40px'}),

        html.Div(id='emotion-image', style={'textAlign': 'center', 'marginTop': '20px'}),

        html.Div(id='insight-plot',
                 style={'maxWidth': '1100px', 'margin': 'auto', 'marginTop': '30px'})
    ], style={
        'fontFamily': 'Arial, sans-serif',
        'backgroundColor': '#F0F9F8',
        'padding': '50px 30px'
    })


In [ ]:
# Main app layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    navbar,
    html.Div(id='page-content'),
    buttons
], style={'maxWidth': '2680px', 'margin': 'auto'})

# Callback to update page content based on URL
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == '/test-emotion':
        return get_test_emotion_content()
    else:  # Default to home page
        return get_home_content(fig1, fig2, fig3, fig4, fig5, fig6)

# Callback for Home Page button
@app.callback(
    Output('url', 'pathname', allow_duplicate=True),
    Input('home-page-btn', 'n_clicks'),
    prevent_initial_call=True
)
def go_to_home(n_clicks):
    if n_clicks:
        return '/'

# Callback for Test Your Emotion button
@app.callback(
    Output('url', 'pathname', allow_duplicate=True),
    Input('test-emotion-btn', 'n_clicks'),
    prevent_initial_call=True
)
def go_to_test_emotion(n_clicks):
    if n_clicks:
        return '/test-emotion'
    
@app.callback(
    Output('prediction-output', 'children'),
    Output('insight-plot', 'children'),
    Output('emotion-image', 'children'), 
    Input('predict-button', 'n_clicks'),
    State('age', 'value'),
    State('gender', 'value'),
    State('platform', 'value'),
    State('daily', 'value'),
    State('posts', 'value'),
    State('likes', 'value'),
    State('comments', 'value'),
    State('messages', 'value'),
    prevent_initial_call=True
)
def predict_emotion(n_clicks, age, gender, platform, daily, posts, likes, comments, messages):
    if n_clicks == 0:
        return "", "", ""

    # Validate inputs
    if any(v is None for v in [age, gender, platform, daily, posts, likes, comments, messages]):
        return "Please fill in all fields", "", ""

    input_data = pd.DataFrame([{
        'Age': age,
        'Gender': gender,
        'Platform': platform,
        'Daily_Usage_Time (minutes)': daily,
        'Posts_Per_Day': posts,
        'Likes_Received_Per_Day': likes,
        'Comments_Received_Per_Day': comments,
        'Messages_Sent_Per_Day': messages
    }])

    prediction = rf_pipeline.predict(input_data)[0]
    print(prediction)
    
    avg_values = df[df['Dominant_Emotion'] == prediction][[
        'Daily_Usage_Time (minutes)', 'Posts_Per_Day', 'Likes_Received_Per_Day',
        'Comments_Received_Per_Day', 'Messages_Sent_Per_Day'
    ]].mean().reset_index()

    avg_values.columns = ['Feature', 'Average']
    avg_values['User Value'] = [daily, posts, likes, comments, messages]

    fig = px.bar(avg_values, x='Feature', y=['Average', 'User Value'],
                 barmode='group',
                 title=f"Your Usage vs Average for {prediction}",
                 color_discrete_sequence=['#2B7A78', '#99D98C'])

    fig.update_layout(
        plot_bgcolor='#E6F7F6',
        paper_bgcolor='#E6F7F6',
        font_color='#17252A'
    )

    # Image handling
    image_filename = f"{prediction}.png"
    
    try:
        # Check if image file exists
        full_image_path = os.path.join('assets', image_filename)
        if os.path.exists(full_image_path):
            img_component = html.Img(src=f"/assets/{prediction}.png", style={"height": "100px"})
        else:
            # Fallback if image doesn't exist
            img_component = html.Div([
                html.P(f"🎭 {prediction}", 
                       style={'fontSize': '48px', 'margin': '0'}),
                html.P("(Image not found)", 
                       style={'fontSize': '12px', 'color': '#888'})
            ])
    except Exception as e:
        # Fallback for any other errors
        img_component = html.Div([
            html.P(f"🎭 {prediction}", 
                   style={'fontSize': '48px', 'margin': '0'})
        ])

    return f"🧠 Predicted Dominant Emotion: {prediction}", dcc.Graph(figure=fig), img_component


# Run the app
port = 8050
print(f"👉 Your dashboard will open at: http://127.0.0.1:{port}/")
app.run(mode='external', port=port)

👉 Your dashboard will open at: http://127.0.0.1:8050/


Neutral
Boredom


# Dashboard (Model)

In [ ]:
rf_pipeline = joblib.load("rf_pipeline.pkl")

In [139]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import os

app = dash.Dash(__name__)
server = app.server  # Needed for deployment

# Assume df and rf_pipeline are defined and loaded properly

app.layout = html.Div([
    html.H2(
        "🎯 Predict Your Dominant Emotion Based on Social Media Usage",
        style={
            'textAlign': 'center',
            'color': '#3AAFA9',
            'marginBottom': '40px',
            'fontWeight': 'bold'
        }
    ),

    html.Div([
        # Left Column
        html.Div([
            html.Label("Age", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='age', type='number', min=10, max=100,
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

            html.Label("Gender", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Dropdown(id='gender', options=[
                {'label': 'Male', 'value': 'Male'},
                {'label': 'Female', 'value': 'Female'}
            ], style={'borderRadius': '5px', 'marginBottom': '15px'}),

            html.Label("Mostly Used Platform", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Dropdown(id='platform', options=[
                {'label': p, 'value': p} for p in df['Platform'].unique()
            ], style={'borderRadius': '5px', 'marginBottom': '15px'}),

            html.Label("Daily Usage Time (minutes)", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='daily', type='number',
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),
        ], style={
            'width': '48%',
            'display': 'inline-block',
            'verticalAlign': 'top',
            'paddingRight': '20px',
            'boxSizing': 'border-box'
        }),

        # Right Column
        html.Div([
            html.Label("Posts Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='posts', type='number',
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

            html.Label("Likes Received Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='likes', type='number',
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

            html.Label("Comments Received Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='comments', type='number',
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),

            html.Label("Messages Sent Per Day", style={'color': '#2B7A78', 'fontWeight': '600'}),
            dcc.Input(id='messages', type='number',
                      style={'width': '100%', 'padding': '10px', 'borderRadius': '5px',
                             'border': '1px solid #95E1D3', 'marginBottom': '15px'}),
        ], style={
            'width': '48%',
            'display': 'inline-block',
            'verticalAlign': 'top'
        }),
    ], style={
        'maxWidth': '1100px',
        'margin': 'auto',
        'backgroundColor': '#E6F7F6',
        'padding': '30px',
        'borderRadius': '12px',
        'boxShadow': '0 4px 12px rgba(0,0,0,0.1)',
        'marginBottom': '40px'
    }),

    html.Div([
        html.Button("Predict Emotion", id='predict-button', n_clicks=0,
                    style={
                        'backgroundColor': '#3AAFA9',
                        'color': 'white',
                        'border': 'none',
                        'padding': '14px 34px',
                        'borderRadius': '25px',
                        'cursor': 'pointer',
                        'fontSize': '16px',
                        'fontWeight': '600',
                        'display': 'block',
                        'margin': '0 auto'
                    }),
    ], style={'marginBottom': '40px'}),

    html.Div(id='prediction-output',
             style={'fontSize': 24, 'textAlign': 'center', 'color': '#17252A', 'fontWeight': 'bold', 'minHeight': '40px'}),

    html.Div(id='emotion-image', style={'textAlign': 'center', 'marginTop': '20px'}),

    html.Div(id='insight-plot',
             style={'maxWidth': '1100px', 'margin': 'auto', 'marginTop': '30px'})
],
    style={
        'fontFamily': 'Arial, sans-serif',
        'backgroundColor': '#F0F9F8',
        'padding': '50px 30px'
    }
)


@app.callback(
    Output('prediction-output', 'children'),
    Output('insight-plot', 'children'),
    Output('emotion-image', 'children'), 
    Input('predict-button', 'n_clicks'),
    State('age', 'value'),
    State('gender', 'value'),
    State('platform', 'value'),
    State('daily', 'value'),
    State('posts', 'value'),
    State('likes', 'value'),
    State('comments', 'value'),
    State('messages', 'value')
)
def predict_emotion(n_clicks, age, gender, platform, daily, posts, likes, comments, messages):
    if n_clicks == 0:
        return "", "", ""

    # Validate inputs
    if any(v is None for v in [age, gender, platform, daily, posts, likes, comments, messages]):
        return "Please fill in all fields", "", ""

    input_data = pd.DataFrame([{
        'Age': age,
        'Gender': gender,
        'Platform': platform,
        'Daily_Usage_Time (minutes)': daily,
        'Posts_Per_Day': posts,
        'Likes_Received_Per_Day': likes,
        'Comments_Received_Per_Day': comments,
        'Messages_Sent_Per_Day': messages
    }])

    prediction = rf_pipeline.predict(input_data)[0]
    print(prediction)
    

    avg_values = df[df['Dominant_Emotion'] == prediction][[
        'Daily_Usage_Time (minutes)', 'Posts_Per_Day', 'Likes_Received_Per_Day',
        'Comments_Received_Per_Day', 'Messages_Sent_Per_Day'
    ]].mean().reset_index()

    avg_values.columns = ['Feature', 'Average']
    avg_values['User Value'] = [daily, posts, likes, comments, messages]

    fig = px.bar(avg_values, x='Feature', y=['Average', 'User Value'],
                 barmode='group',
                 title=f"Your Usage vs Average for {prediction}",
                 color_discrete_sequence=['#2B7A78', '#99D98C'])

    fig.update_layout(
        plot_bgcolor='#E6F7F6',
        paper_bgcolor='#E6F7F6',
        font_color='#17252A'
    )

    # FIXED: Correct image path construction for Dash
    # Remove the leading slash - Dash serves assets from 'assets/' not '/assets/'
    image_filename = f"{prediction}.png"
    image_path = f"assets/{image_filename}"  # Changed from "/assets/" to "assets/"
    
    # Additional error handling for image loading
    try:
        # Check if image file exists (optional)
        full_image_path = os.path.join('assets', image_filename)
        if os.path.exists(full_image_path):
            img_component = html.Img(src=f"/assets/{prediction}.png", style={"height": "100px"})

        else:
            # Fallback if image doesn't exist
            img_component = html.Div([
                html.P(f"🎭 {prediction}", 
                       style={'fontSize': '48px', 'margin': '0'}),
                html.P("(Image not found)", 
                       style={'fontSize': '12px', 'color': '#888'})
            ])
    except Exception as e:
        # Fallback for any other errors
        img_component = html.Div([
            html.P(f"🎭 {prediction}", 
                   style={'fontSize': '48px', 'margin': '0'})
        ])

    return f"🧠 Predicted Dominant Emotion: {prediction}", dcc.Graph(figure=fig), img_component


if __name__ == '__main__':
    app.run(debug=True)

Boredom
Boredom
Happiness
